# 1.- Definición General


## intención del colab

La diferencia con el colab de  dataset básico es que voy a dejar


*   completas las series con 0 ventas en el medio
*   dejar el dataset por cliente producto partido por clusters


#### Incluye
*   Datos básicos del problema
*   Datos de insights del problema
*   Datos de Argentina (sin adivinar futuro)



#### NO incluye
*   Deltas, lags
*   relaciones estadísticas: medias, desvíos, min, max, ponderados y periódicos
*   Correlaciones entre series
*   Trabajos con NA / ceros





# 2.- Datos básicos del problema df_base_30

### Ambito general

In [1]:
### me preparo en el entorno
# entorno
ColabNotebook = 'google.colab' in str(get_ipython())

if ColabNotebook:
    # monta G-drive en entorno COLAB
    from google.colab import drive
    drive.mount('/content/drive/')

    # carpeta donde se encuentran archivos .py auxiliares
    #FUENTES_DIR = '/content/drive/MyDrive/Colab Notebooks/FUENTES/'
    #DATOS_DIR = '/content/drive/MyDrive/Colab Notebooks/DATOS/'      # carpeta donde se encuentran los datasets
    FUENTES_DIR = '/content/drive/MyDrive/MCD/Labo3/FUENTES/' # Ajustar según usuario
    DATOS_DIR = '/content/drive/MyDrive/MCD_grupo17_la_desgracia/lab3/PruebasAK/DATOSL3/' # ajustar según usuario OJO crear un acceso directo para poder "montar" una unidad compartida
else:
    # configuración para notebook con instalación LOCAL
    # FUENTES_DIR = '../FUENTES'         # carpeta donde se encuentran archivos .py auxiliares
    DATOS_DIR   = './DATOS/' # carpeta donde se encuentran los datasets

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

Mounted at /content/drive/


In [2]:
## Cargo lo necesario
# Cargo paquetes
#Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from numpy.polynomial import Polynomial
#from scipy.signal import detrend

#import holidays

#Configuración de parametros de Pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.precision', 3)
pd.set_option('plotting.backend', 'matplotlib')

### Funciones útiles, gracias Diana Bermudez


In [3]:
def eliminar_columnas_solo_ceros(df: pd.DataFrame) -> pd.DataFrame:
    """
    Elimina columnas de un DataFrame que contienen exclusivamente el valor cero.

    Args:
        df (pd.DataFrame): El DataFrame de entrada con múltiples columnas.

    Returns:
        pd.DataFrame: Un nuevo DataFrame con las columnas que no eran solo ceros.
                      Si no hay columnas restantes, devuelve un DataFrame vacío.
    """
    # Identificar las columnas que contienen solo ceros
    # Para cada columna, verificamos si la suma de sus valores absolutos es cero.
    # Esto maneja tanto ceros enteros como ceros flotantes.
    columnas_a_mantener = [col for col in df.columns if df[col].abs().sum() != 0]

    # Crear un nuevo DataFrame solo con las columnas a mantener
    df_resultante = df[columnas_a_mantener].copy()

    return df_resultante

In [4]:
def reduce_mem_usage(df):
    """Itera por las columnas del DataFrame y modifica el tipo de datos para reducir uso de memoria."""
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Uso de memoria inicial del DataFrame: {start_mem:.2f} MB')

    for col in df.columns:
        col_type = df[col].dtype

        if pd.api.types.is_numeric_dtype(col_type):
            c_min = df[col].min()
            c_max = df[col].max()

            if pd.api.types.is_integer_dtype(col_type):
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            # Sólo convertir a categoría si no lo es ya
            if not pd.api.types.is_categorical_dtype(df[col]):
                df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Uso de memoria final del DataFrame: {end_mem:.2f} MB')
    print(f'Memoria reducida en un {(100 * (start_mem - end_mem) / start_mem):.2f}%')
    return df

### Carga de archivos datos

In [17]:
#cargar csv
# ventas al canal, va a ser el archivo padre
sellin = pd.read_csv(DATOS_DIR +'sell-in.txt', sep='\t')
# productos a validar, por ahora no se me ocurre como usarlos, pero los cargo por las dudas. Son los que se van a estar vivos
productos_validar = pd.read_csv(DATOS_DIR +'product_id_apredecir201912.txt', sep='\t')
# cargo la tabla de características
prod_cat = pd.read_csv(DATOS_DIR +'tb_productos.txt', sep='\t')
# cargo los stocks de los productos
stocks = pd.read_csv(DATOS_DIR +'tb_stocks.txt', sep='\t')
# me traigo el clasificador de clusters
df_llave= pd.read_csv(DATOS_DIR +'df_llave_cluster_93.csv')

### proceso básico de sell-in => df_base_1

In [18]:
print(sellin.shape)
sellin.head()

(2945818, 7)


,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.053,0.053
1,201701,10032,20524,0,1,0.136,0.136
2,201701,10217,20524,0,1,0.030,0.030
3,201701,10125,20524,0,1,0.023,0.023
4,201701,10012,20524,0,11,1.545,1.545


In [19]:
# lo primero que hago es eliminar cust_request_qty porque debe er la relación entre peso y cust request tn
# y si dejo cust_request_tn va a ser demasiando igual a tn, por lo que voy a dejar solamente dif_tn y dif_por como las diferencias absolutas y porcentuales sobre lo pedido
df_base_1 = sellin.drop(columns=['cust_request_qty'])
df_base_1['dif_tn'] = df_base_1['tn'] - df_base_1['cust_request_tn']
df_base_1['dif_por'] = df_base_1['dif_tn'] / df_base_1['cust_request_tn']
df_base_1 = df_base_1.drop(columns=['cust_request_tn'])
df_base_1.head()

,periodo,customer_id,product_id,plan_precios_cuidados,tn,dif_tn,dif_por
0,201701,10234,20524,0,0.053,0.0,0.0
1,201701,10032,20524,0,0.136,0.0,0.0
2,201701,10217,20524,0,0.030,0.0,0.0
3,201701,10125,20524,0,0.023,0.0,0.0
4,201701,10012,20524,0,1.545,0.0,0.0


In [20]:
# como no sé bien dónde va a caer precios cuidados cuando complete con ceros y como a nadie le dió importante
# dropeo la columna de precios cuidados
df_base_1 = df_base_1.drop(columns=['plan_precios_cuidados'])
df_base_1.head()

,periodo,customer_id,product_id,tn,dif_tn,dif_por
0,201701,10234,20524,0.053,0.0,0.0
1,201701,10032,20524,0.136,0.0,0.0
2,201701,10217,20524,0.030,0.0,0.0
3,201701,10125,20524,0.023,0.0,0.0
4,201701,10012,20524,1.545,0.0,0.0


In [21]:
## de la columna periodo tengo que generar mes, año, trimestre, date(formato daytime), secuencia que es el mes consecutivo
## genero también una columna date_mes en formato período que me facilite la vida más adelante.
df_base_1['anio']=df_base_1['periodo'] //100
df_base_1['mes']=df_base_1['periodo'] % 100
df_base_1['secuencia']= (df_base_1['anio']-2017)*12+df_base_1['mes']
df_base_1['trimestre']=df_base_1['mes']//3
df_base_1['date'] = pd.to_datetime(df_base_1['periodo'], format='%Y%m')
df_base_1['date_mes'] = df_base_1['date'].dt.to_period('M')
df_base_1.head()


,periodo,customer_id,product_id,tn,dif_tn,dif_por,anio,mes,secuencia,trimestre,date,date_mes
0,201701,10234,20524,0.053,0.0,0.0,2017,1,1,0,2017-01-01,2017-01
1,201701,10032,20524,0.136,0.0,0.0,2017,1,1,0,2017-01-01,2017-01
2,201701,10217,20524,0.030,0.0,0.0,2017,1,1,0,2017-01-01,2017-01
3,201701,10125,20524,0.023,0.0,0.0,2017,1,1,0,2017-01-01,2017-01
4,201701,10012,20524,1.545,0.0,0.0,2017,1,1,0,2017-01-01,2017-01


In [22]:
# para simplificar voy a eliminar las columnas anio mes secuencia trimestre date periodo
df_base_1 = df_base_1.drop(columns=['anio', 'mes', 'secuencia', 'trimestre', 'date', 'periodo'])
df_base_1.head()

,customer_id,product_id,tn,dif_tn,dif_por,date_mes
0,10234,20524,0.053,0.0,0.0,2017-01
1,10032,20524,0.136,0.0,0.0,2017-01
2,10217,20524,0.030,0.0,0.0,2017-01
3,10125,20524,0.023,0.0,0.0,2017-01
4,10012,20524,1.545,0.0,0.0,2017-01


In [23]:
# veo los types
df_base_1.dtypes

,0
customer_id,int64
product_id,int64
tn,float64
dif_tn,float64
dif_por,float64
date_mes,period[M]


In [24]:
df_base_1.head()

,customer_id,product_id,tn,dif_tn,dif_por,date_mes
0,10234,20524,0.053,0.0,0.0,2017-01
1,10032,20524,0.136,0.0,0.0,2017-01
2,10217,20524,0.030,0.0,0.0,2017-01
3,10125,20524,0.023,0.0,0.0,2017-01
4,10012,20524,1.545,0.0,0.0,2017-01


### ahora quiero completar los ceros de productos  y  customer

Tengo una matriz larga de ventas con muchas columnas. Entre ellas tengo customer_id que identifica el cliente y product_id que identifica el producto. Además tengo una columna date formato YYYY-MM que refleja la fecha de la venta, una columna tn que es la venta y una serie de columnas cuyo valor está relacionado al producto y una serie de columnas cuyo valor está relacionado al cliente. Son las ventas realmente efectuadas. Para el análisis siguiente tengo que ser capaz de distinguir si una venta no se produjo porque el cliente o el producto no existían o porque el cliente decidió no comprar. Por ello tengo que realizar las siguientes operaciones. Voy a considerar que un cliente está activo a partir de su primera compra. Y que un producto está activo a partir de su primera venta. Necesito entonces llenar el df existente completando las ventas de 0 tn de todo producto activo para todo cliente activo que no lo haya comprado en cada período. parafrasear para verificar entendimiento y generar código python comentado

In [25]:
df_base_1.dtypes

,0
customer_id,int64
product_id,int64
tn,float64
dif_tn,float64
dif_por,float64
date_mes,period[M]


In [26]:
# 1. Determina la primera y última fecha de compra de cada cliente
first_customer_date = df_base_1.groupby('customer_id')['date_mes'].min().rename('first_customer_date')
last_customer_date = df_base_1.groupby('customer_id')['date_mes'].max().rename('last_customer_date')

In [27]:
# 2. Determina la primera y última fecha de venta de cada producto
first_product_date = df_base_1.groupby('product_id')['date_mes'].min().rename('first_product_date')
last_product_date = df_base_1.groupby('product_id')['date_mes'].max().rename('last_product_date')

In [28]:
# 3. Consigue todos los meses disponibles
all_periods = df_base_1['date_mes'].sort_values().unique()

In [29]:
# 4. Saca todas las combinaciones posibles (producto, cliente, mes)
clientes = df_base_1['customer_id'].unique()
productos = df_base_1['product_id'].unique()
grid = pd.MultiIndex.from_product(
    [clientes, productos, all_periods],
    names=['customer_id', 'product_id', 'date_mes']
)
full_df = pd.DataFrame(index=grid).reset_index()

In [30]:
# 5. Añade las fechas de activación y último registro
full_df = (full_df
    .merge(first_customer_date, on='customer_id')
    .merge(last_customer_date,  on='customer_id')
    .merge(first_product_date, on='product_id')
    .merge(last_product_date,  on='product_id')
)

In [31]:
# 6. Filtra SOLO donde ambos están activos en ese mes
mask = (
    (full_df['date_mes'] >= full_df['first_customer_date']) &
    (full_df['date_mes'] >= full_df['first_product_date'])
)
full_df = full_df[mask].copy()

In [32]:
# 7. Une tu base original para saber cuándo hubo venta
full_df = full_df.merge(
    df_base_1[['customer_id', 'product_id', 'date_mes', 'tn', 'dif_tn', 'dif_por']],
    on=['customer_id', 'product_id', 'date_mes'],
    how='left'
)

In [33]:
# 8. Donde no hay venta, rellena con cero
for col in ['tn', 'dif_tn', 'dif_por']:
    full_df[col] = full_df[col].fillna(0)

In [34]:
full_df.shape

(19639107, 10)

In [35]:
df_base_1.shape

(2945818, 6)

In [36]:
full_df.head()

,customer_id,product_id,date_mes,first_customer_date,last_customer_date,first_product_date,last_product_date,tn,dif_tn,dif_por
0,10234,20524,2017-01,2017-01,2019-12,2017-01,2019-12,0.053,0.0,0.0
1,10234,20524,2017-02,2017-01,2019-12,2017-01,2019-12,0.000,0.0,0.0
2,10234,20524,2017-03,2017-01,2019-12,2017-01,2019-12,0.015,0.0,0.0
3,10234,20524,2017-04,2017-01,2019-12,2017-01,2019-12,0.000,0.0,0.0
4,10234,20524,2017-05,2017-01,2019-12,2017-01,2019-12,0.000,0.0,0.0


In [37]:
# agrego el tiempo de vida del cliente
primer_mes = full_df['date_mes'].min()

def calcular_vida_prod(row):
    # Si el producto existe desde el primer mes, aplicar mínimo de 4
    if row['first_product_date'] == primer_mes:
        vida = (row['date_mes'] - row['first_product_date']).n + 4
    else:
        vida = (row['date_mes'] - row['first_product_date']).n + 1  # sumar 1 para contar el mes inicial
    return max(vida, 1)  # Solo por seguridad, nunca sea menor que 1

full_df['vida_prod'] = full_df.apply(calcular_vida_prod, axis=1)
full_df.head()

,customer_id,product_id,date_mes,first_customer_date,last_customer_date,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod
0,10234,20524,2017-01,2017-01,2019-12,2017-01,2019-12,0.053,0.0,0.0,4
1,10234,20524,2017-02,2017-01,2019-12,2017-01,2019-12,0.000,0.0,0.0,5
2,10234,20524,2017-03,2017-01,2019-12,2017-01,2019-12,0.015,0.0,0.0,6
3,10234,20524,2017-04,2017-01,2019-12,2017-01,2019-12,0.000,0.0,0.0,7
4,10234,20524,2017-05,2017-01,2019-12,2017-01,2019-12,0.000,0.0,0.0,8


In [38]:
full_df.dtypes

,0
customer_id,int64
product_id,int64
date_mes,period[M]
first_customer_date,period[M]
last_customer_date,period[M]
first_product_date,period[M]
last_product_date,period[M]
tn,float64
dif_tn,float64
dif_por,float64


In [39]:
# Función para calcular la vida del cliente
def calcular_vida_customer(row):
    # Si el cliente existe desde el primer mes, aplicar mínimo de 4 (como para producto)
    if row['first_customer_date'] == primer_mes:
        vida = (row['date_mes'] - row['first_customer_date']).n + 4
    else:
        vida = (row['date_mes'] - row['first_customer_date']).n + 1  # suma 1 para incluir el mes inicial
    return max(vida, 1)

# Aplica la función fila a fila
full_df['vida_customer'] = full_df.apply(calcular_vida_customer, axis=1)

# Ahora tu df tiene ambas columnas: vida_prod y vida_customer
full_df.head()

,customer_id,product_id,date_mes,first_customer_date,last_customer_date,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,vida_customer
0,10234,20524,2017-01,2017-01,2019-12,2017-01,2019-12,0.053,0.0,0.0,4,4
1,10234,20524,2017-02,2017-01,2019-12,2017-01,2019-12,0.000,0.0,0.0,5,5
2,10234,20524,2017-03,2017-01,2019-12,2017-01,2019-12,0.015,0.0,0.0,6,6
3,10234,20524,2017-04,2017-01,2019-12,2017-01,2019-12,0.000,0.0,0.0,7,7
4,10234,20524,2017-05,2017-01,2019-12,2017-01,2019-12,0.000,0.0,0.0,8,8


In [40]:
full_df.tail()

,customer_id,product_id,date_mes,first_customer_date,last_customer_date,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,vida_customer
19639102,10572,21054,2019-12,2019-12,2019-12,2019-10,2019-12,0.0,0.0,0.0,3,1
19639103,10572,20728,2019-12,2019-12,2019-12,2019-11,2019-12,0.0,0.0,0.0,2,1
19639104,10572,20792,2019-12,2019-12,2019-12,2019-12,2019-12,0.0,0.0,0.0,1,1
19639105,10572,20854,2019-12,2019-12,2019-12,2019-12,2019-12,0.0,0.0,0.0,1,1
19639106,10572,20770,2019-12,2019-12,2019-12,2019-12,2019-12,0.0,0.0,0.0,1,1


In [41]:
# Asume full_df ya está ordenado por customer_id, product_id, date_mes
full_df = full_df.sort_values(['customer_id', 'product_id', 'date_mes']).reset_index(drop=True)

# Creamos columnas vacías
full_df['meses_on'] = 0
full_df['meses_off'] = 0

# Calculamos el primer período global
primer_mes = full_df['date_mes'].min()

# Función que aplica la lógica mencionada sobre cada grupo cliente-producto
def acumular_on_off(grp):
    meses_on = []
    meses_off = []
    on_count = 0
    off_count = 0
    for i, row in grp.iterrows():
        # Primer período
        if row['date_mes'] == primer_mes:
            if row['tn'] != 0:
                on_count = 4
                off_count = 0
            else:
                on_count = 0
                off_count = 4
        else:
            if row['tn'] != 0:
                on_count = on_count + 1 if on_count > 0 else 1
                off_count = 0
            else:
                on_count = 0
                off_count = off_count + 1 if off_count > 0 else 1
        meses_on.append(on_count)
        meses_off.append(off_count)
    grp['meses_on'] = meses_on
    grp['meses_off'] = meses_off
    return grp

# Aplicamos la función a cada grupo de cliente-producto
full_df = full_df.groupby(['customer_id', 'product_id'], group_keys=False).apply(acumular_on_off)

# ¡Listo! Ahora full_df tiene las columnas meses_on y meses_off con la lógica indicada.

/tmp/ipython-input-41-4161351945.py:40: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  full_df = full_df.groupby(['customer_id', 'product_id'], group_keys=False).apply(acumular_on_off)


In [42]:
full_df.head()

,customer_id,product_id,date_mes,first_customer_date,last_customer_date,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,vida_customer,meses_on,meses_off
0,10001,20001,2017-01,2017-01,2019-12,2017-01,2019-12,99.439,0.000,0.000,4,4,4,0
1,10001,20001,2017-02,2017-01,2019-12,2017-01,2019-12,198.844,0.000,0.000,5,5,5,0
2,10001,20001,2017-03,2017-01,2019-12,2017-01,2019-12,92.465,0.000,0.000,6,6,6,0
3,10001,20001,2017-04,2017-01,2019-12,2017-01,2019-12,13.297,0.000,0.000,7,7,7,0
4,10001,20001,2017-05,2017-01,2019-12,2017-01,2019-12,101.006,-0.201,-0.002,8,8,8,0


In [43]:
# 1. Creamos un set para búsqueda eficiente
productos_indeterminados = set(productos_validar['product_id'].values)

# 2. Función para asignar vida_restante por fila
def calcular_vida_restante(row):
    if row['product_id'] in productos_indeterminados:
        return 60
    else:
        # (asumiendo que las columnas son period[M])
        return max(0, (row['last_product_date'] - row['date_mes']).n)
        # max(0, ...) por si date_mes supera last_product_date nunca sea negativo

# 3. Aplicamos la función fila a fila
full_df['vida_restante'] = full_df.apply(calcular_vida_restante, axis=1)

# Listo, full_df ahora tiene la columna vida_restante según tus reglas.

In [44]:
full_df.head()

,customer_id,product_id,date_mes,first_customer_date,last_customer_date,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,vida_customer,meses_on,meses_off,vida_restante
0,10001,20001,2017-01,2017-01,2019-12,2017-01,2019-12,99.439,0.000,0.000,4,4,4,0,60
1,10001,20001,2017-02,2017-01,2019-12,2017-01,2019-12,198.844,0.000,0.000,5,5,5,0,60
2,10001,20001,2017-03,2017-01,2019-12,2017-01,2019-12,92.465,0.000,0.000,6,6,6,0,60
3,10001,20001,2017-04,2017-01,2019-12,2017-01,2019-12,13.297,0.000,0.000,7,7,7,0,60
4,10001,20001,2017-05,2017-01,2019-12,2017-01,2019-12,101.006,-0.201,-0.002,8,8,8,0,60


In [45]:
# Lista de columnas calculadas
cols = [
    'first_customer_date', 'last_customer_date',
    'first_product_date', 'last_product_date',
    'vida_prod', 'vida_customer', 'meses_on', 'meses_off', 'vida_restante'
]

# Mostramos la cantidad de valores únicos por columna calculada
for col in cols:
    print(f'{col}: {full_df[col].nunique()} valores únicos')

first_customer_date: 33 valores únicos
last_customer_date: 34 valores únicos
first_product_date: 33 valores únicos
last_product_date: 35 valores únicos
vida_prod: 39 valores únicos
vida_customer: 39 valores únicos
meses_on: 40 valores únicos
meses_off: 40 valores únicos
vida_restante: 37 valores únicos


In [46]:
full_df.describe()

,customer_id,product_id,tn,dif_tn,dif_por,vida_prod,vida_customer,meses_on,meses_off,vida_restante
count,1.964e+07,1.964e+07,1.964e+07,1.964e+07,1.964e+07,1.964e+07,1.964e+07,1.964e+07,1.964e+07,1.964e+07
mean,1.029e+04,2.059e+04,6.747e-02,-1.430e-03,-1.968e-04,1.995e+01,2.150e+01,5.130e-01,1.041e+01,4.064e+01
std,1.761e+02,3.693e+02,1.215e+00,1.457e-01,1.044e-02,1.076e+01,1.060e+01,2.293e+00,1.049e+01,2.563e+01
min,1.000e+04,2.000e+04,0.000e+00,-1.406e+02,-9.996e-01,1.000e+00,1.000e+00,0.000e+00,0.000e+00,0.000e+00
25%,1.014e+04,2.027e+04,0.000e+00,0.000e+00,0.000e+00,1.100e+01,1.300e+01,0.000e+00,2.000e+00,1.100e+01
50%,1.028e+04,2.055e+04,0.000e+00,0.000e+00,0.000e+00,2.000e+01,2.200e+01,0.000e+00,7.000e+00,6.000e+01
75%,1.043e+04,2.088e+04,0.000e+00,0.000e+00,0.000e+00,2.900e+01,3.100e+01,0.000e+00,1.700e+01,6.000e+01
max,1.064e+04,2.130e+04,5.479e+02,1.519e+01,1.040e+01,3.900e+01,3.900e+01,3.900e+01,3.900e+01,6.000e+01


In [47]:
# genero una columna anio como integrer a partir de date_mes
full_df['anio'] = full_df['date_mes'].dt.year
full_df['mes'] = full_df['date_mes'].dt.month
full_df.head()

,customer_id,product_id,date_mes,first_customer_date,last_customer_date,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,vida_customer,meses_on,meses_off,vida_restante,anio,mes
0,10001,20001,2017-01,2017-01,2019-12,2017-01,2019-12,99.439,0.000,0.000,4,4,4,0,60,2017,1
1,10001,20001,2017-02,2017-01,2019-12,2017-01,2019-12,198.844,0.000,0.000,5,5,5,0,60,2017,2
2,10001,20001,2017-03,2017-01,2019-12,2017-01,2019-12,92.465,0.000,0.000,6,6,6,0,60,2017,3
3,10001,20001,2017-04,2017-01,2019-12,2017-01,2019-12,13.297,0.000,0.000,7,7,7,0,60,2017,4
4,10001,20001,2017-05,2017-01,2019-12,2017-01,2019-12,101.006,-0.201,-0.002,8,8,8,0,60,2017,5


In [48]:
# genero la columna secuencia
full_df['secuencia']= (full_df['anio']-2017)*12+full_df['mes']
full_df['trimestre']=full_df['date_mes'].dt.quarter
full_df.head()

,customer_id,product_id,date_mes,first_customer_date,last_customer_date,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,vida_customer,meses_on,meses_off,vida_restante,anio,mes,secuencia,trimestre
0,10001,20001,2017-01,2017-01,2019-12,2017-01,2019-12,99.439,0.000,0.000,4,4,4,0,60,2017,1,1,1
1,10001,20001,2017-02,2017-01,2019-12,2017-01,2019-12,198.844,0.000,0.000,5,5,5,0,60,2017,2,2,1
2,10001,20001,2017-03,2017-01,2019-12,2017-01,2019-12,92.465,0.000,0.000,6,6,6,0,60,2017,3,3,1
3,10001,20001,2017-04,2017-01,2019-12,2017-01,2019-12,13.297,0.000,0.000,7,7,7,0,60,2017,4,4,2
4,10001,20001,2017-05,2017-01,2019-12,2017-01,2019-12,101.006,-0.201,-0.002,8,8,8,0,60,2017,5,5,2


In [49]:
full_df.shape

(19639107, 19)

### datos básico categoría de productos y luego unión para df_base_2


In [50]:
# busco y elimino registros duplicados
# Supongamos que tu columna se llama 'columna'
duplicados = prod_cat['product_id'].duplicated().any()
print("¿Hay duplicados?:", duplicados)
duplicados_valores = prod_cat['product_id'][prod_cat['product_id'].duplicated()]
print("Valores duplicados:")
print(duplicados_valores.unique())

¿Hay duplicados?: True
Valores duplicados:
[20010 20022 20021 20020 20278 20192 20037 20105 20100 20623 20230]


In [51]:
#veo los duplicados a ver si amerita algún tratamiento o son simple error
duplicadas_filas = prod_cat[prod_cat['product_id'].duplicated(keep=False)]
print(duplicadas_filas)

      cat1            cat2            cat3     brand  sku_size  product_id
36      HC     ROPA LAVADO           Polvo   LIMPIEX       400       20010
37      HC     ROPA LAVADO           Polvo   LIMPIEX       400       20010
38      HC     ROPA LAVADO           Polvo   LIMPIEX       800       20022
39      HC     ROPA LAVADO           Polvo   LIMPIEX       800       20022
41      HC     ROPA LAVADO           Polvo   LIMPIEX       400       20021
42      HC     ROPA LAVADO           Polvo   LIMPIEX       400       20021
43      HC     ROPA LAVADO           Polvo   LIMPIEX       800       20020
44      HC     ROPA LAVADO           Polvo   LIMPIEX       800       20020
541  FOODS  SOPAS Y CALDOS      Caldo Cubo     MAGGI         2       20278
542  FOODS  SOPAS Y CALDOS      Caldo Cubo     MAGGI         2       20278
550  FOODS  SOPAS Y CALDOS      Caldo Cubo     MAGGI         2       20192
551  FOODS  SOPAS Y CALDOS      Caldo Cubo     MAGGI         2       20192
552  FOODS  SOPAS Y CALDO

In [52]:
prod_cat.isnull().sum()

,0
cat1,0
cat2,0
cat3,0
brand,0
sku_size,0
product_id,0


In [53]:
#elimino los duplicados
# el peor caso es un shampoo por un acondicionador... no cambia mucho
prod_cat_unic = prod_cat.drop_duplicates(subset=['product_id'])
duplicados = prod_cat_unic['product_id'].duplicated().any()
print("¿Hay duplicados?:", duplicados)
duplicados_valores = prod_cat_unic['product_id'][prod_cat_unic['product_id'].duplicated()]
print("Valores duplicados:")
print(duplicados_valores.unique())

¿Hay duplicados?: False
Valores duplicados:
[]


In [54]:
#voy metiendo todo en una sola matriz,
# que me llevo
columnas_a_agregar = ['cat1', 'cat2', 'cat3','brand','sku_size', 'product_id']
# resultado
df_base_2 = full_df.merge(prod_cat_unic[columnas_a_agregar], on='product_id', how='left')

df_base_2.head()

,customer_id,product_id,date_mes,first_customer_date,last_customer_date,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,vida_customer,meses_on,meses_off,vida_restante,anio,mes,secuencia,trimestre,cat1,cat2,cat3,brand,sku_size
0,10001,20001,2017-01,2017-01,2019-12,2017-01,2019-12,99.439,0.000,0.000,4,4,4,0,60,2017,1,1,1,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
1,10001,20001,2017-02,2017-01,2019-12,2017-01,2019-12,198.844,0.000,0.000,5,5,5,0,60,2017,2,2,1,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
2,10001,20001,2017-03,2017-01,2019-12,2017-01,2019-12,92.465,0.000,0.000,6,6,6,0,60,2017,3,3,1,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
3,10001,20001,2017-04,2017-01,2019-12,2017-01,2019-12,13.297,0.000,0.000,7,7,7,0,60,2017,4,4,2,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
4,10001,20001,2017-05,2017-01,2019-12,2017-01,2019-12,101.006,-0.201,-0.002,8,8,8,0,60,2017,5,5,2,HC,ROPA LAVADO,Liquido,ARIEL,3000.0


In [55]:
df_base_2.shape

(19639107, 24)

In [56]:
df_base_2.isnull().sum()

,0
customer_id,0
product_id,0
date_mes,0
first_customer_date,0
last_customer_date,0
first_product_date,0
last_product_date,0
tn,0
dif_tn,0
dif_por,0


In [57]:
# tengo el problema que hay 782000 instancias donde no puedo asociar producto con características
# posiblement los productos que fueron mal colocados
# los voy a reemplazar por UNKN por desconocido
columnas_raras =['cat1', 'cat2', 'cat3','brand','sku_size']
df_base_2[columnas_raras] = df_base_2[columnas_raras].fillna('UNKN')
df_base_2.isnull().sum()
#

,0
customer_id,0
product_id,0
date_mes,0
first_customer_date,0
last_customer_date,0
first_product_date,0
last_product_date,0
tn,0
dif_tn,0
dif_por,0


# 3.- Datos de insights del problema

### en clase se habló de ver hermanos mayores y menores, mismo producto distinto size, no queda muy claro como hacerlo a partir de la codificación, por ahora no lo hago


### la estructura de categorías, el árbol no me resulta claro, por lo que voy a crear nuevas categorías que lo reflejen df_base_3


In [58]:
# cuando yo voy partiendo en categorías la idea es ir quedandome con subconjuntos lo más parecidos posibles
# y nada me garantiza la unicidad o correspondencia de las catagorías que aquí aparecen más abajo
# por ejemplo puedo tener un jabón de 500 ml  para bebé y uno de lavar la ropa de 500ml, no me hace sentido agruparlos solamente por su size
# entonces voy a generar Categorías a partir de la 2 que sean una concatenación de los valores de más arriba en la jerarquia
df_base_3=df_base_2.copy()
df_base_3['cat1_cat2'] = df_base_3['cat1'].fillna('') + '_' + df_base_3['cat2'].fillna('')
df_base_3['cat1_cat2_cat3'] = df_base_3['cat1_cat2'].fillna('') + '_' + df_base_3['cat3'].fillna('')
df_base_3['cat1_cat2_cat3_brand'] = df_base_3['cat1_cat2_cat3'].fillna('') + '_' + df_base_3['brand'].fillna('')
df_base_3['cat1_cat2_cat3_brand_size'] = df_base_3['cat1_cat2_cat3_brand'].fillna('') + '_' + df_base_3['sku_size'].astype(str)
df_base_3.head()

,customer_id,product_id,date_mes,first_customer_date,last_customer_date,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,vida_customer,meses_on,meses_off,vida_restante,anio,mes,secuencia,trimestre,cat1,cat2,cat3,brand,sku_size,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size
0,10001,20001,2017-01,2017-01,2019-12,2017-01,2019-12,99.439,0.000,0.000,4,4,4,0,60,2017,1,1,1,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0
1,10001,20001,2017-02,2017-01,2019-12,2017-01,2019-12,198.844,0.000,0.000,5,5,5,0,60,2017,2,2,1,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0
2,10001,20001,2017-03,2017-01,2019-12,2017-01,2019-12,92.465,0.000,0.000,6,6,6,0,60,2017,3,3,1,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0
3,10001,20001,2017-04,2017-01,2019-12,2017-01,2019-12,13.297,0.000,0.000,7,7,7,0,60,2017,4,4,2,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0
4,10001,20001,2017-05,2017-01,2019-12,2017-01,2019-12,101.006,-0.201,-0.002,8,8,8,0,60,2017,5,5,2,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0


In [59]:
# saco las columnas que ahora son redundantes
df_base_3 = df_base_3.drop(columns=[ 'cat2', 'cat3','brand','sku_size'])
df_base_3.head()

,customer_id,product_id,date_mes,first_customer_date,last_customer_date,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,vida_customer,meses_on,meses_off,vida_restante,anio,mes,secuencia,trimestre,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size
0,10001,20001,2017-01,2017-01,2019-12,2017-01,2019-12,99.439,0.000,0.000,4,4,4,0,60,2017,1,1,1,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0
1,10001,20001,2017-02,2017-01,2019-12,2017-01,2019-12,198.844,0.000,0.000,5,5,5,0,60,2017,2,2,1,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0
2,10001,20001,2017-03,2017-01,2019-12,2017-01,2019-12,92.465,0.000,0.000,6,6,6,0,60,2017,3,3,1,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0
3,10001,20001,2017-04,2017-01,2019-12,2017-01,2019-12,13.297,0.000,0.000,7,7,7,0,60,2017,4,4,2,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0
4,10001,20001,2017-05,2017-01,2019-12,2017-01,2019-12,101.006,-0.201,-0.002,8,8,8,0,60,2017,5,5,2,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0


In [60]:
# agrego stock a fin de mes por producto, no me interesa en esta instancia generar los deltas , medias etc)
print(stocks.tail())

       periodo  product_id  stock_final
13686   201912       20453        1.437
13687   201912       21026        7.268
13688   201912       21054        0.508
13689   201912       20981        2.185
13690   201912       20853        1.824


In [61]:
# tengo que reconstruir la columna periodo para poder hacer el merge
# defino periodo para df_base_3 como un integrer que es el anio multiplicado por 100 más el mes
df_base_3['periodo'] = df_base_3['anio'] * 100 + df_base_3['mes']
df_base_3.head()

,customer_id,product_id,date_mes,first_customer_date,last_customer_date,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,vida_customer,meses_on,meses_off,vida_restante,anio,mes,secuencia,trimestre,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,periodo
0,10001,20001,2017-01,2017-01,2019-12,2017-01,2019-12,99.439,0.000,0.000,4,4,4,0,60,2017,1,1,1,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201701
1,10001,20001,2017-02,2017-01,2019-12,2017-01,2019-12,198.844,0.000,0.000,5,5,5,0,60,2017,2,2,1,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201702
2,10001,20001,2017-03,2017-01,2019-12,2017-01,2019-12,92.465,0.000,0.000,6,6,6,0,60,2017,3,3,1,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201703
3,10001,20001,2017-04,2017-01,2019-12,2017-01,2019-12,13.297,0.000,0.000,7,7,7,0,60,2017,4,4,2,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201704
4,10001,20001,2017-05,2017-01,2019-12,2017-01,2019-12,101.006,-0.201,-0.002,8,8,8,0,60,2017,5,5,2,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201705


In [62]:
# 2. Añadir el stock. La clave de unión es el mes Y el producto.
df_base_3 = pd.merge(df_base_3, stocks, on=['periodo', 'product_id'], how='left')
df_base_3.tail()

,customer_id,product_id,date_mes,first_customer_date,last_customer_date,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,vida_customer,meses_on,meses_off,vida_restante,anio,mes,secuencia,trimestre,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,periodo,stock_final
19639102,10637,21299,2019-08,2017-09,2017-09,2017-08,2017-08,0.0,0.0,0.0,25,24,0,24,0,2019,8,32,3,UNKN,UNKN_UNKN,UNKN_UNKN_UNKN,UNKN_UNKN_UNKN_UNKN,UNKN_UNKN_UNKN_UNKN_UNKN,201908,NaN
19639103,10637,21299,2019-09,2017-09,2017-09,2017-08,2017-08,0.0,0.0,0.0,26,25,0,25,0,2019,9,33,3,UNKN,UNKN_UNKN,UNKN_UNKN_UNKN,UNKN_UNKN_UNKN_UNKN,UNKN_UNKN_UNKN_UNKN_UNKN,201909,NaN
19639104,10637,21299,2019-10,2017-09,2017-09,2017-08,2017-08,0.0,0.0,0.0,27,26,0,26,0,2019,10,34,4,UNKN,UNKN_UNKN,UNKN_UNKN_UNKN,UNKN_UNKN_UNKN_UNKN,UNKN_UNKN_UNKN_UNKN_UNKN,201910,NaN
19639105,10637,21299,2019-11,2017-09,2017-09,2017-08,2017-08,0.0,0.0,0.0,28,27,0,27,0,2019,11,35,4,UNKN,UNKN_UNKN,UNKN_UNKN_UNKN,UNKN_UNKN_UNKN_UNKN,UNKN_UNKN_UNKN_UNKN_UNKN,201911,NaN
19639106,10637,21299,2019-12,2017-09,2017-09,2017-08,2017-08,0.0,0.0,0.0,29,28,0,28,0,2019,12,36,4,UNKN,UNKN_UNKN,UNKN_UNKN_UNKN,UNKN_UNKN_UNKN_UNKN,UNKN_UNKN_UNKN_UNKN_UNKN,201912,NaN


In [63]:
df_base_3.to_parquet(DATOS_DIR +'df_base_3_prod_cli.parquet', index=False)

In [66]:
df_base_3.dtypes


,0
customer_id,int64
product_id,int64
date_mes,period[M]
first_customer_date,period[M]
last_customer_date,period[M]
first_product_date,period[M]
last_product_date,period[M]
tn,float64
dif_tn,float64
dif_por,float64


## Ahora voy a agregar los datos de argentina

In [64]:
df_base_20 =pd.read_csv(DATOS_DIR +'df_base_20.csv')
df_base_20.head()

,periodo,date,date_mes,dias_laborables_arg,cisne_negro,VtaGral,VtaGralCte,Bebidas,Almacén,Panadería,Lácteos,Carnes,Verdulería y frutería,Artículos de limpieza y perfumería,"Indumentaria, calzado y textiles para el hogar",Electrónicos y artículos para el hogar,Otros,USD_of,USD_blue,Infl,Expect
0,201701,2017-01-01 00:00:00,2017-01,22,0,27858.87,27477.31,573064.06,1.624e+06,11576.02,253462.37,73883.84,20142.56,1.219e+06,1146.38,48728.12,269322.15,15.7,16.47,1.6,44.47
1,201702,2017-02-01 00:00:00,2017-02,18,0,27858.87,27477.31,573064.06,1.624e+06,11576.02,253462.37,73883.84,20142.56,1.219e+06,1146.38,48728.12,269322.15,15.7,16.24,1.6,40.68
2,201703,2017-03-01 00:00:00,2017-03,22,0,25994.20,25223.43,540122.06,1.559e+06,11395.77,252720.27,75545.85,20456.38,1.089e+06,1018.56,34128.35,257899.92,15.3,15.88,2.1,40.95
3,201704,2017-04-01 00:00:00,2017-04,18,0,27780.01,26334.26,586230.17,2.006e+06,12889.83,307951.54,89305.98,22607.84,1.296e+06,927.07,36321.56,301861.90,15.2,15.59,2.4,46.19
4,201705,2017-05-01 00:00:00,2017-05,21,0,29134.27,27045.88,572661.71,1.967e+06,13642.25,298638.09,84722.68,24054.43,1.219e+06,1098.13,43054.66,298741.07,15.2,15.99,2.7,45.76


In [65]:
# elimino las columnas date y date_mes de df_base_20
df_base_20 = df_base_20.drop(columns=['date', 'date_mes'])
df_base_20.head()

,periodo,dias_laborables_arg,cisne_negro,VtaGral,VtaGralCte,Bebidas,Almacén,Panadería,Lácteos,Carnes,Verdulería y frutería,Artículos de limpieza y perfumería,"Indumentaria, calzado y textiles para el hogar",Electrónicos y artículos para el hogar,Otros,USD_of,USD_blue,Infl,Expect
0,201701,22,0,27858.87,27477.31,573064.06,1.624e+06,11576.02,253462.37,73883.84,20142.56,1.219e+06,1146.38,48728.12,269322.15,15.7,16.47,1.6,44.47
1,201702,18,0,27858.87,27477.31,573064.06,1.624e+06,11576.02,253462.37,73883.84,20142.56,1.219e+06,1146.38,48728.12,269322.15,15.7,16.24,1.6,40.68
2,201703,22,0,25994.20,25223.43,540122.06,1.559e+06,11395.77,252720.27,75545.85,20456.38,1.089e+06,1018.56,34128.35,257899.92,15.3,15.88,2.1,40.95
3,201704,18,0,27780.01,26334.26,586230.17,2.006e+06,12889.83,307951.54,89305.98,22607.84,1.296e+06,927.07,36321.56,301861.90,15.2,15.59,2.4,46.19
4,201705,21,0,29134.27,27045.88,572661.71,1.967e+06,13642.25,298638.09,84722.68,24054.43,1.219e+06,1098.13,43054.66,298741.07,15.2,15.99,2.7,45.76


In [67]:
df_base_20.dtypes

,0
periodo,int64
dias_laborables_arg,int64
cisne_negro,int64
VtaGral,float64
VtaGralCte,float64
Bebidas,float64
Almacén,float64
Panadería,float64
Lácteos,float64
Carnes,float64


In [68]:
df_base_3 = df_base_3.merge(df_base_20, on='periodo', how='left')

In [70]:
df_base_3.head()

,customer_id,product_id,date_mes,first_customer_date,last_customer_date,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,vida_customer,meses_on,meses_off,vida_restante,anio,mes,secuencia,trimestre,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,periodo,stock_final,dias_laborables_arg,cisne_negro,VtaGral,VtaGralCte,Bebidas,Almacén,Panadería,Lácteos,Carnes,Verdulería y frutería,Artículos de limpieza y perfumería,"Indumentaria, calzado y textiles para el hogar",Electrónicos y artículos para el hogar,Otros,USD_of,USD_blue,Infl,Expect
0,10001,20001,2017-01,2017-01,2019-12,2017-01,2019-12,99.439,0.000,0.000,4,4,4,0,60,2017,1,1,1,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201701,NaN,22,0,27858.87,27477.31,573064.06,1.624e+06,11576.02,253462.37,73883.84,20142.56,1.219e+06,1146.38,48728.12,269322.15,15.7,16.47,1.6,44.47
1,10001,20001,2017-02,2017-01,2019-12,2017-01,2019-12,198.844,0.000,0.000,5,5,5,0,60,2017,2,2,1,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201702,NaN,18,0,27858.87,27477.31,573064.06,1.624e+06,11576.02,253462.37,73883.84,20142.56,1.219e+06,1146.38,48728.12,269322.15,15.7,16.24,1.6,40.68
2,10001,20001,2017-03,2017-01,2019-12,2017-01,2019-12,92.465,0.000,0.000,6,6,6,0,60,2017,3,3,1,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201703,NaN,22,0,25994.20,25223.43,540122.06,1.559e+06,11395.77,252720.27,75545.85,20456.38,1.089e+06,1018.56,34128.35,257899.92,15.3,15.88,2.1,40.95
3,10001,20001,2017-04,2017-01,2019-12,2017-01,2019-12,13.297,0.000,0.000,7,7,7,0,60,2017,4,4,2,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201704,NaN,18,0,27780.01,26334.26,586230.17,2.006e+06,12889.83,307951.54,89305.98,22607.84,1.296e+06,927.07,36321.56,301861.90,15.2,15.59,2.4,46.19
4,10001,20001,2017-05,2017-01,2019-12,2017-01,2019-12,101.006,-0.201,-0.002,8,8,8,0,60,2017,5,5,2,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201705,NaN,21,0,29134.27,27045.88,572661.71,1.967e+06,13642.25,298638.09,84722.68,24054.43,1.219e+06,1098.13,43054.66,298741.07,15.2,15.99,2.7,45.76


In [71]:
df_llave.head()

,cluster_id,product_cust_id,std_ventas
0,1,20002_10001,110.699
1,2,20001_10001,106.033
2,2,20001_10004,114.273
3,3,20001_10007,51.573
4,1,20002_10007,54.293


In [73]:
# en df_base_3 genero una columna product_cust_id haciendo product_id +"_" + customer_id
df_base_3['product_cust_id'] = df_base_3['product_id'].astype(str) + "_" + df_base_3['customer_id'].astype(str)
df_base_3.head()


,customer_id,product_id,date_mes,first_customer_date,last_customer_date,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,vida_customer,meses_on,meses_off,vida_restante,anio,mes,secuencia,trimestre,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,periodo,stock_final,dias_laborables_arg,cisne_negro,VtaGral,VtaGralCte,Bebidas,Almacén,Panadería,Lácteos,Carnes,Verdulería y frutería,Artículos de limpieza y perfumería,"Indumentaria, calzado y textiles para el hogar",Electrónicos y artículos para el hogar,Otros,USD_of,USD_blue,Infl,Expect,product_cust_id
0,10001,20001,2017-01,2017-01,2019-12,2017-01,2019-12,99.439,0.000,0.000,4,4,4,0,60,2017,1,1,1,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201701,NaN,22,0,27858.87,27477.31,573064.06,1.624e+06,11576.02,253462.37,73883.84,20142.56,1.219e+06,1146.38,48728.12,269322.15,15.7,16.47,1.6,44.47,20001_10001
1,10001,20001,2017-02,2017-01,2019-12,2017-01,2019-12,198.844,0.000,0.000,5,5,5,0,60,2017,2,2,1,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201702,NaN,18,0,27858.87,27477.31,573064.06,1.624e+06,11576.02,253462.37,73883.84,20142.56,1.219e+06,1146.38,48728.12,269322.15,15.7,16.24,1.6,40.68,20001_10001
2,10001,20001,2017-03,2017-01,2019-12,2017-01,2019-12,92.465,0.000,0.000,6,6,6,0,60,2017,3,3,1,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201703,NaN,22,0,25994.20,25223.43,540122.06,1.559e+06,11395.77,252720.27,75545.85,20456.38,1.089e+06,1018.56,34128.35,257899.92,15.3,15.88,2.1,40.95,20001_10001
3,10001,20001,2017-04,2017-01,2019-12,2017-01,2019-12,13.297,0.000,0.000,7,7,7,0,60,2017,4,4,2,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201704,NaN,18,0,27780.01,26334.26,586230.17,2.006e+06,12889.83,307951.54,89305.98,22607.84,1.296e+06,927.07,36321.56,301861.90,15.2,15.59,2.4,46.19,20001_10001
4,10001,20001,2017-05,2017-01,2019-12,2017-01,2019-12,101.006,-0.201,-0.002,8,8,8,0,60,2017,5,5,2,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201705,NaN,21,0,29134.27,27045.88,572661.71,1.967e+06,13642.25,298638.09,84722.68,24054.43,1.219e+06,1098.13,43054.66,298741.07,15.2,15.99,2.7,45.76,20001_10001


### ahora creo los clusters

In [74]:
# 1. Unimos la columna cluster_id de df_llave a df_base_3 usando product_cust_id
df_merged = df_base_3.merge(df_llave[['product_cust_id', 'cluster_id']],
                            on='product_cust_id',
                            how='left')

# 2. Creamos los DataFrames según corresponda
df_cluster_1 = df_merged[df_merged['cluster_id'] == 1].copy()
df_cluster_2 = df_merged[df_merged['cluster_id'] == 2].copy()
df_cluster_3 = df_merged[df_merged['cluster_id'] == 3].copy()

# 3. El DataFrame con los product_cust_id que NO aparecen en df_llave (cluster_id es NaN tras el merge)
df_cluster_4 = df_merged[df_merged['cluster_id'].isna()].copy()

In [76]:
# dropeo la columnas clustr_id de df_cluster_1; df_cluster_2; df_cluster_3 ; df_cluster_4
df_cluster_1 = df_cluster_1.drop(columns=['cluster_id'])
df_cluster_2 = df_cluster_2.drop(columns=['cluster_id'])
df_cluster_3 = df_cluster_3.drop(columns=['cluster_id'])
df_cluster_4 = df_cluster_4.drop(columns=['cluster_id'])


In [77]:
df_cluster_1.shape

(236511, 45)

In [78]:
df_cluster_2.shape

(629073, 45)

In [79]:
df_cluster_3.shape

(174785, 45)

In [80]:
df_cluster_4.shape

(18598738, 45)

In [81]:
df_cluster_1.to_parquet(DATOS_DIR +'df_cluster_1.parquet', index=False)
df_cluster_2.to_parquet(DATOS_DIR +'df_cluster_2.parquet', index=False)
df_cluster_3.to_parquet(DATOS_DIR +'df_cluster_3.parquet', index=False)
df_cluster_4.to_parquet(DATOS_DIR +'df_cluster_4.parquet', index=False)